<a href="https://colab.research.google.com/github/yangr296/ECE539/blob/main/ECE539_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
#import data from drive
import tensorflow as tf
import numpy as np
import random
from google.colab import drive
from sklearn.model_selection import train_test_split


drive.mount('/content/drive')
with open('/content/drive/Shared drives/ECE539/parkinsons/parkinsons.data', 'r') as f:
  tmp = np.genfromtxt(f,delimiter=',')
  np.set_printoptions(precision=3,suppress=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Preprocessing

In [128]:
from tensorflow import keras

X, y = tmp[1:, 1:], tmp[1:, -7]
X = np.delete(X, 16, 1)
X = X.astype(float)
"""
index by columns
X[0] MDVP:Fo(Hz)
X[1] MDVP:Fhi(Hz)
X[2] MDVP:Flo(Hz)
X[3] MDVP:Jitter(%)
X[4] MDVP:Jitter(Abs)
X[5] MDVP:RAP
X[6] MDVP:PPQ
X[7] Jitter:DDP
X[8] MDVP:Shimmer
X[9] MDVP:Shimmer(dB)
X[10] Shimmer:APQ3
X[11] Shimmer:APQ5
X[12] MDVP:APQ
X[13] Shimmer:DDA
X[14] NHR
X[15] HNR
X[16] RPDE
X[17] DFA
X[18] spread1
X[19] spread2
X[20] D2
X[21] PPE
"""

(195, 22)


'\nindex by columns\nX[0] MDVP:Fo(Hz)\nX[1] MDVP:Fhi(Hz)\nX[2] MDVP:Flo(Hz)\nX[3] MDVP:Jitter(%)\nX[4] MDVP:Jitter(Abs)\nX[5] MDVP:RAP\nX[6] MDVP:PPQ\nX[7] Jitter:DDP\nX[8] MDVP:Shimmer\nX[9] MDVP:Shimmer(dB)\nX[10] Shimmer:APQ3\nX[11] Shimmer:APQ5\nX[12] MDVP:APQ\nX[13] Shimmer:DDA\nX[14] NHR\nX[15] HNR\nX[16] RPDE\nX[17] DFA\nX[18] spread1\nX[19] spread2\nX[20] D2\nX[21] PPE\n'

In [146]:
from scipy.stats import pearsonr
Corr = [[0] * 2 for i in range(22)]
for i in range(22):
  feature = X[:, i]
  corr, _ = pearsonr(feature, y)
  Corr[i][0] = abs(corr)
  Corr[i][1] = i
  print("Pearson's Correlation of feature", i, "is: %.3f" % corr)

Pearson's Correlation of feature 0 is: -0.384
Pearson's Correlation of feature 1 is: -0.166
Pearson's Correlation of feature 2 is: -0.380
Pearson's Correlation of feature 3 is: 0.278
Pearson's Correlation of feature 4 is: 0.339
Pearson's Correlation of feature 5 is: 0.267
Pearson's Correlation of feature 6 is: 0.289
Pearson's Correlation of feature 7 is: 0.267
Pearson's Correlation of feature 8 is: 0.367
Pearson's Correlation of feature 9 is: 0.351
Pearson's Correlation of feature 10 is: 0.348
Pearson's Correlation of feature 11 is: 0.351
Pearson's Correlation of feature 12 is: 0.364
Pearson's Correlation of feature 13 is: 0.348
Pearson's Correlation of feature 14 is: 0.189
Pearson's Correlation of feature 15 is: -0.362
Pearson's Correlation of feature 16 is: 0.309
Pearson's Correlation of feature 17 is: 0.232
Pearson's Correlation of feature 18 is: 0.565
Pearson's Correlation of feature 19 is: 0.455
Pearson's Correlation of feature 20 is: 0.340
Pearson's Correlation of feature 21 is: 

In [153]:
# sort all the correlations
num = 22
Corr.sort()
print("ranking the %i lowest correlations: " %num)
for i in range(num):
  print("feature: %2d corr: %.3f" %(Corr[i][1], Corr[i][0]))

ranking the 22 lowest correlations: 
feature:  1 corr: 0.166
feature: 14 corr: 0.189
feature: 17 corr: 0.232
feature:  7 corr: 0.267
feature:  5 corr: 0.267
feature:  3 corr: 0.278
feature:  6 corr: 0.289
feature: 16 corr: 0.309
feature:  4 corr: 0.339
feature: 20 corr: 0.340
feature: 13 corr: 0.348
feature: 10 corr: 0.348
feature:  9 corr: 0.351
feature: 11 corr: 0.351
feature: 15 corr: 0.362
feature: 12 corr: 0.364
feature:  8 corr: 0.367
feature:  2 corr: 0.380
feature:  0 corr: 0.384
feature: 19 corr: 0.455
feature: 21 corr: 0.531
feature: 18 corr: 0.565


In [ ]:
# partition into training and validation set at 80/20 division
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# Converting y_r and y_t into one-hot encoding using a keras utility
#  to_categorical

# print(y_test)
#[1, 0] is 0
#[0, 1] is 1
y_test = keras.utils.to_categorical(y_test)
y_train = keras.utils.to_categorical(y_train)
# print(y_test)

In [118]:
from sklearn.impute import KNNImputer

if np.sum(np.isnan(X)):
  print('Total of NaN before imputation:', np.sum(np.isnan(X)))
  imputer = KNNImputer(n_neighbors=2, weights="uniform")
  X1 = imputer.fit_transform(X)
  print('Total of NaN after imputation:', np.sum(np.isnan(X1)))
else:
  X1 = X
  print('no NaN')

no NaN


In [119]:
from sklearn import preprocessing
print("before transformation\n--------------------")
for i in range(X.shape[1]):
  print("range of feature %i is %i"%(i, np.ptp(X_train[:,i])))
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
print("after transformation\n--------------------")
for i in range(X.shape[1]):
  print("range of feature %i is %i"%(i, np.ptp(X_train[:,i])))

before transformation
--------------------
range of feature 0 is 156
range of feature 1 is 489
range of feature 2 is 173
range of feature 3 is 0
range of feature 4 is 0
range of feature 5 is 0
range of feature 6 is 0
range of feature 7 is 0
range of feature 8 is 0
range of feature 9 is 0
range of feature 10 is 0
range of feature 11 is 0
range of feature 12 is 0
range of feature 13 is 0
range of feature 14 is 0
range of feature 15 is 24
range of feature 16 is 0
range of feature 17 is 0
range of feature 18 is 5
range of feature 19 is 0
range of feature 20 is 2
range of feature 21 is 0
after transformation
--------------------
range of feature 0 is 2
range of feature 1 is 2
range of feature 2 is 2
range of feature 3 is 2
range of feature 4 is 2
range of feature 5 is 2
range of feature 6 is 2
range of feature 7 is 2
range of feature 8 is 1
range of feature 9 is 2
range of feature 10 is 1
range of feature 11 is 2
range of feature 12 is 2
range of feature 13 is 1
range of feature 14 is 1
ran

Model Definition

In [120]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_dim=22, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')])

In [121]:
# compile the keras model
net.compile(loss='binary_crossentropy', optimizer='Adadelta',
              metrics=['accuracy'])

Training

In [122]:
# fit the keras model on the dataset
net.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
16/16 [==============================] - 1s 2ms/step - loss: 0.8706 - accuracy: 0.4808
Epoch 2/150
16/16 [==============================] - 0s 2ms/step - loss: 0.9062 - accuracy: 0.5192
Epoch 3/150
16/16 [==============================] - 0s 2ms/step - loss: 0.8308 - accuracy: 0.5256
Epoch 4/150
16/16 [==============================] - 0s 3ms/step - loss: 0.8697 - accuracy: 0.5192
Epoch 5/150
16/16 [==============================] - 0s 3ms/step - loss: 0.8772 - accuracy: 0.5449
Epoch 6/150
16/16 [==============================] - 0s 3ms/step - loss: 0.8538 - accuracy: 0.4808
Epoch 7/150
16/16 [==============================] - 0s 2ms/step - loss: 0.8555 - accuracy: 0.4872
Epoch 8/150
16/16 [==============================] - 0s 2ms/step - loss: 0.9014 - accuracy: 0.5128
Epoch 9/150
16/16 [==============================] - 0s 5ms/step - loss: 0.8716 - accuracy: 0.5449
Epoch 10/150
16/16 [==============================] - 0s 4ms/step - loss: 0.8372 - accuracy: 0.5192
Epoch 11/

Testing

In [123]:
# Evaluate the trained model using keras built-in function
score = net.evaluate(X_test, y_test, verbose=0)
print("Test loss:", format(score[0],".4f"))
print("Test accuracy:", score[1])

Test loss: 0.7358
Test accuracy: 0.5641025900840759
